In [ ]:
!pip install arcgis==2.3.0
!pip install torch==2.0.1 torchvision==0.15.2 fastai==1.0.61 transformers==4.36.2 optuna==3.0.4
!pip install timm==0.4.12

In [ ]:
!pip install fastai==1.0.61

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from arcgis.learn import prepare_textdata
from arcgis.learn.text import SequenceToSequence

In [ ]:
input_csv = '/content/FHA Single Family Servicing Guide – 4000.1- Published 01-18-2023 - FINAL UPLOAD.csv'
df = pd.read_csv(input_csv, low_memory=False)

In [ ]:
df.head()

,Question,Sub Question,Compliance Item
0,Was the cost of subservicing imposed on the bo...,NaN,The costs associated with subservicing may not...
1,Was the cost of subservicing passed along to H...,NaN,The costs associated with subservicing may not...
2,Did the Transferor Servicing Mortgagee verify ...,NaN,The Transferor Servicing Mortgagee must verify...
3,"On the Transfer Date, did the Transferee Servi...",NaN,"On the Transfer Date, the Transferee Servicing..."
4,"On the Transfer Date, did the Transferee Servi...",NaN,"On the Transfer Date, the Transferee Servicing..."


In [ ]:
# Combine 'Question' and 'Sub Question' into 'Question' column
df['Question'] = df['Question'].fillna('')  # Ensure no NaN, just empty strings
df['Sub Question'] = df['Sub Question'].fillna('')  # Ensure no NaN

# If 'Question' is blank, use 'Sub Question' value, and vice versa
df['Question'] = df['Question'].where(df['Question'] != '', df['Sub Question'])
df['Question'] = df['Question'].combine_first(df['Sub Question'])

# Drop 'Sub Question' column if no longer needed
df = df.drop(columns=['Sub Question'])

# Show the updated DataFrame
df.head()

,Question,Compliance Item
0,Was the cost of subservicing imposed on the bo...,The costs associated with subservicing may not...
1,Was the cost of subservicing passed along to H...,The costs associated with subservicing may not...
2,Did the Transferor Servicing Mortgagee verify ...,The Transferor Servicing Mortgagee must verify...
3,"On the Transfer Date, did the Transferee Servi...","On the Transfer Date, the Transferee Servicing..."
4,"On the Transfer Date, did the Transferee Servi...","On the Transfer Date, the Transferee Servicing..."


In [ ]:
# Define the size of the test set (e.g., 30%)
test_size = 0.3

# Split the DataFrame into training and test sets
train_df, test_df = train_test_split(df, test_size=test_size, random_state=42)

# Save the training and test sets to separate CSV files
train_csv = '/content/train.csv'
test_csv = '/content/test.csv'

train_df.to_csv(train_csv, index=False)
test_df.to_csv(test_csv, index=False)

In [ ]:
data = prepare_textdata(path='/content/', batch_size=4, task='sequence_translation',
                        text_columns='Compliance Item', label_columns='Question',
                        train_file='ques_gen_data_from_compliance.csv')

In [ ]:
data.show_batch()

source,target
The Eligible Depository the Servicer selects must honor sight drafts Freddie Mac may present at any,Did the Eligible Depository the Servicer selected honor sight drafts Freddie Mac might present at any time against any Custodial Account and/or related Time Deposit that the Servicer maintained?
"C. Submission/Completion Of Form HUD-300. A Form HUD-300, Monthly Summary of Assistance Payments Due","Did the Form HUD-300, Monthly Summary of Assistance Payments due under Sections 235(b), 235(J) or 235(i), or Interest Reduction Payment due, under Section 236, accompany the completed Form HUD-93102?"
Fannie Mae will reimburse the servicer for costs related to â€¢ preparing documentation for a comple,Was the servicer reimbursed by Fannie Mae for the costs related to preparing documentation for a completed Mortgage Release?
"When reporting, ensure that the: DDLPI reflects the due date of the last fully paid installment","When mortgage was reported as paid in full, was it ensured that DDLPI reflected the due date of the last fully paid installment?"
"The transferor servicer must deliver to the transferee servicer : Trial balances, as of the close of","Did the transferor servicer deliver to the transferee servicer Trial balances, as of the close of business on the day immediately preceding the transfer date showing transfers of ownership, payoffs, and other exception transactions that are in process, including mortgage loan modification-related transactions?"


In [ ]:
SequenceToSequence.supported_backbones

['T5', 'Bart', 'Marian']

In [ ]:
SequenceToSequence.available_backbone_models('T5')

['t5-small',
 't5-base',
 't5-large',
 't5-3b',
 't5-11b',
 'See all T5 models at https://huggingface.co/models?filter=t5 ']

In [ ]:
model = SequenceToSequence(data,backbone='t5-base')

RecursionError: maximum recursion depth exceeded

In [ ]:
lr = model.lr_find()

NameError: name 'model' is not defined

In [ ]:
lr

NameError: name 'lr' is not defined

In [ ]:
model.fit(epochs=6, lr=lr, early_stopping=True)

NameError: name 'lr' is not defined

In [ ]:
model.get_model_metrics()

{'seq2seq_acc': 0.9424, 'bleu': 0.9004}

In [ ]:
model.show_results()

text,target,pred
"When the Servicer receives notice, or confirms information that a Borrower has filed a bankruptcy petition, at a minimum, the Servicer must:1. Comply with all applicable laws and regulations, including working with the debtor's pre-bankruptcy credit counseling agency on a debt management plan, if applicable 2. Obtain a copy of the Borrower's (debtor's) bankruptcy petition or other bankruptcy notice 3. Accurately complete and file a proof of claim, including all required proof of claim forms, within the time limitations set by the bankruptcy court. This may include, but is not limited to, providing timely information, documentation (e.g., Borrower payment history) and payoff and reinstatement figures necessary for bankruptcy counsel to meet the time limits set by the bankruptcy court if counsel is representing the Servicer in the case. If additional costs are incurred after confirmation of the bankruptcy plan, work with the trustee to ensure all steps are taken to recover those costs allowable by law from the debtor through the plan, if applicable. The Servicer must, prior to filing any proof of claim or motion for relief from the stay with respect to a Mortgage registered on the MERS® System, prepare and execute (using the Servicer's employee who is a MERS authorized â“signing officerâ) an assignment of the Security Instrument from MERS to the Servicer. The Servicer must record the prepared assignment where required by State law. State mandated recordings are non-reimbursable by Freddie Mac, are not considered part of the Freddie Mac allowable foreclosure counsel fees and must not be billed to the Borrower. 4. Monitor the bankruptcy filing and obtain status of the proceedings from the trustee in a timely manner. This includes, at a minimum, assisting with any motions for relief of stay and monitoring the first meeting of creditors, proof of claim and/or confirmation of the bankruptcy plan, pre-and post-petition payments, pleadings and notices. If counsel requests additional documentation and/or information from the Servicer, at any time, the Servicer must provide such requested information and/or documents within three Business Days after receipt of the request, or within such earlier time frame if necessary, to comply with timing requirements under applicable law or court orders and procedures. 5. Maintain copies","For a leasehold mortgage, when the Servicer received notice, or confirmed information that a Borrower had filed a bankruptcy petition, at a minimum, did the Servicer Object to any Chapter 13 plan that did not provide for payment of the ground rent in addition to mortgage payments and real estate taxes?","When the Servicer received notice, or confirmed information that a Borrower had filed a bankruptcy petition, at a minimum, did the Servicer accurately complete and file a proof of claim, including all required proof of claim forms"
"When determining eligibility for a forbearance plan for a borrower impacted by COVID-19, the property securing the mortgage loan may be either a principal residence, a second home, or an investment property. The servicer must otherwise follow the requirements in Servicing Guide D2-3.2-01, Forbearance Plan. In response to the CARES Act, the servicer must approve forbearance plans for borrowers impacted by COVID-19 in accordance with the CARES Act. The CARES Act states that a forbearance plan must be provided to any borrower who requests a forbearance with an attestation of the financial hardship caused by the COVID-19 emergency; and no additional documentation other than the borrowerâ€TMs attestation to a financial hardship caused by the COVID-19 emergency is required. Such a borrower must be provided an initial forbearance plan for a period up to 180 days, and that forbearance period may be extended for up to an additional 180 days at the request of the borrower. In accordance with D2-3.2-01, the servicer may provide an initial forbearance period, and any extended forb

In [ ]:
model.save('arc_ques_4E_bleu_90')

Computing model metrics...


PosixPath('/content/models/arc_ques_4E_bleu_90')

In [ ]:
txt = [' The Mortgagee must ensure that the Borrower meets the following eligibility criteria for the FHA-HAMP Option: The Borrower has not executed an FHA-HAMP agreement in the past 24 months. Presidentially-Declared Major Disaster Area (PDMDA) Disaster Loss Mitigation Options and COVID-19 Home Retention Options do not count against this requirement. ']

In [ ]:
model.predict(txt)

[(' The Mortgagee must ensure that the Borrower meets the following eligibility criteria for the FHA-HAMP Option: The Borrower has not executed an FHA-HAMP agreement in the past 24 months. Presidentially-Declared Major Disaster Area (PDMDA) Disaster Loss Mitigation Options and COVID-19 Home Retention Options do not count against this requirement. ',
  'Did the Mortgagee ensure that the Borrower met the eligibility criteria for the FHA-')]

In [ ]:
def predict_and_store_results(csv_path, model):
    # Load the validation data
    val_data = pd.read_csv(csv_path)

    # Extract input and target texts
    inputs = val_data['COMPL_Compliance']
    targets = val_data['CSTEPQ_Question']

    # Apply the prediction function to the input_text column
    predictions = [model.predict(text) for text in inputs]

    # Create a DataFrame with the original inputs, targets, and predictions
    result_df = pd.DataFrame({
        "input_texts": inputs,
        "target_texts": targets,
        "predicted_text": predictions
    })

    return result_df

In [ ]:
result_df = predict_and_store_results('/content/val.csv', model)

In [ ]:
result_df.to_csv('/content/results.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/arc_ques_fnma_bleu_97.zip

Archive:  /content/arc_ques_fnma_bleu_97.zip
warning [/content/arc_ques_fnma_bleu_97.zip]:  2097152 extra bytes at beginning or within zipfile
  (attempting to process anyway)
file #1:  bad zipfile offset (local header sig):  2097152
  (attempting to re-compensate)
   creating: arc_ques_fnma_bleu_97/
error: invalid zip file with overlapped components (possible zip bomb)


In [ ]:
seq2seqmodel = SequenceToSequence.from_model('/content/arc_ques_fnma_bleu_97')

`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.


RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
txt =
'''

'''

In [ ]:
seq2seqmodel.predict(txt, max_length=100)